<a href="https://colab.research.google.com/github/gyuwonh-ai/SeSAC_miniProject_1Team/blob/main/hundred/Hundred_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hundred 자세의 기준을 잡기위한 EDA

In [ ]:
# 1. 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from scipy.stats import linregress


# --- 2. 분석 함수 정의 ---
def calculate_hundred_metrics(df):
    """
    Hundred 지표 (각도, 높이)를 계산
    - (Smart Leg) 신뢰도 높은 다리 자동 선택
    - (abs(dx)) 방향에 상관없이 예각(Angle) 계산
    """
    df_out = df.copy()

    # (1) 다리 선택 (잘 보이는 쪽으로)
    # 원본 _conf 컬럼이 있는지 확인
    if 'Left_Ankle_conf' not in df.columns or 'Right_Ankle_conf' not in df.columns:
        print("    ⚠️ Ankle_conf 컬럼이 없어 '왼쪽 다리'를 기준으로 계산합니다.")
        use_left_leg_mask = True # conf 없으면 왼쪽 기준
    else:
        use_left_leg_mask = df['Left_Ankle_conf'] >= df['Right_Ankle_conf']

    smart_ankle_x = np.where(use_left_leg_mask, df['norm_Left_Ankle_x'], df['norm_Right_Ankle_x'])
    smart_ankle_y = np.where(use_left_leg_mask, df['norm_Left_Ankle_y'], df['norm_Right_Ankle_y'])
    smart_hip_x = np.where(use_left_leg_mask, df['norm_Left_Hip_x'], df['norm_Right_Hip_x'])
    smart_hip_y = np.where(use_left_leg_mask, df['norm_Left_Hip_y'], df['norm_Right_Hip_y'])

    # (1_A) 다리 높이 각도 (Smart Leg 기준)
    dy = -(smart_ankle_y - smart_hip_y) # Y축 반전
    dx = np.abs(smart_ankle_x - smart_hip_x) # [수정] 절댓값 적용
    df_out['Leg_Height_Angle'] = np.degrees(np.arctan2(dy, dx))

    # (2) 코어 고정각 (Shoulder-Hip-Knee)
    def get_angle(a, b, c):
        ba = a - b
        bc = c - b
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

    p_shoulder = df[['norm_Left_Shoulder_x', 'norm_Left_Shoulder_y']].values
    p_hip = df[['norm_Left_Hip_x', 'norm_Left_Hip_y']].values
    p_knee = df[['norm_Left_Knee_x', 'norm_Left_Knee_y']].values

    df_out['Core_Angle'] = [get_angle(s, h, k) if not (np.isnan(s).any() or np.isnan(h).any() or np.isnan(k).any()) else np.nan
                            for s, h, k in zip(p_shoulder, p_hip, p_knee)]

    # (3) 상체 컬업 (Nose가 Shoulder Center보다 얼마나 위에 있나)
    shoulder_center_y = (df['norm_Left_Shoulder_y'] + df['norm_Right_Shoulder_y']) / 2
    df_out['Head_Curl_Position'] = shoulder_center_y - df['norm_Nose_y']

    return df_out

def analyze_stability(group):
    """
    하나의 영상(group)을 받아 안정성 지표를 계산
    [업그레이드!] 자세의 "가장 긴 연속 구간(Longest Streak)"을 'Active'로 감지
    """

    # --- [보간(Interpolation) 전처리] ---
    # (이 부분은 이전과 동일)
    parts_to_clean = ['Left_Ankle', 'Right_Ankle', 'Left_Hip', 'Right_Hip',
                      'Left_Shoulder', 'Right_Shoulder', 'Nose', 'Left_Knee']
    group_clean = group.copy()
    for part in parts_to_clean:
        if f'{part}_conf' in group_clean.columns:
            low_conf_mask = group_clean[f'{part}_conf'] < 0.5
            group_clean.loc[low_conf_mask, f'norm_{part}_x'] = np.nan
            group_clean.loc[low_conf_mask, f'norm_{part}_y'] = np.nan
    norm_cols = [col for col in group_clean.columns if col.startswith('norm_')]
    group_clean[norm_cols] = group_clean[norm_cols].interpolate(method='linear', limit_direction='both', limit=5)
    # --- [전처리 끝] ---

    # 1. 'Hundred' 지표 계산
    group_with_metrics = calculate_hundred_metrics(group_clean) # (이 함수는 이전에 정의됨)
    group_with_metrics = group_with_metrics.dropna(subset=['Leg_Height_Angle', 'Head_Curl_Position'])

    if group_with_metrics.empty:
        return None

    # --- [핵심 수정] Longest Streak 로직 ---
    # (1) "후보 프레임" 찾기 (물리적 조건)
    group_with_metrics['is_candidate'] = (
        (group_with_metrics['Leg_Height_Angle'] > 5) &
        (group_with_metrics['Leg_Height_Angle'] < 45) &
        (group_with_metrics['Head_Curl_Position'] > 0.05)
    )

    # (2) 연속된 "True" 블록(Streak)에 ID 부여하기
    # (is_candidate 값이 바뀔 때마다 cumsum()으로 새 ID를 만듦)
    group_with_metrics['block_id'] = (group_with_metrics['is_candidate'] != group_with_metrics['is_candidate'].shift()).cumsum()

    # (3) "True" 블록(후보 구간)들의 길이(프레임 수) 계산
    candidate_streaks = group_with_metrics[group_with_metrics['is_candidate'] == True].groupby('block_id').size()

    if candidate_streaks.empty:
        # 'True'인 블록이 하나도 없음 (운동 안 함)
        return None

    # (4) "가장 긴" 블록(Streak)의 ID 찾기
    longest_block_id = candidate_streaks.idxmax()

    # (5) "가장 긴" 블록의 프레임들만 'Active'로 정의
    active_frames = group_with_metrics[
        (group_with_metrics['block_id'] == longest_block_id) &
        (group_with_metrics['is_candidate'] == True)
    ].copy()
    # --- [수정 완료] ---

    if len(active_frames) < 10: # 가장 긴 구간도 10프레임 미만이면 무시
        return None

    # (2) 지표 계산: 흔들림 (표준편차)
    stability_score = active_frames['Leg_Height_Angle'].std()
    core_stability = active_frames['Core_Angle'].std()

    # (3) 지표 계산: 다리 처짐 (기울기)
    if len(active_frames['Frame'].unique()) > 1:
        slope, _ = np.polyfit(active_frames['Frame'], active_frames['Leg_Height_Angle'], 1)
    else:
        slope = 0

    start_frame = active_frames['Frame'].min()
    end_frame = active_frames['Frame'].max()

    return pd.Series({
        'Leg_Stability_StdDev': stability_score,
        'Core_Stability_StdDev': core_stability,
        'Leg_Fatigue_Slope': slope,
        'Avg_Leg_Angle': active_frames['Leg_Height_Angle'].mean(),
        'Avg_Core_Angle': active_frames['Core_Angle'].mean(),
        'Active_Frame_Count': len(active_frames),
        'Active_Start_Frame': start_frame,
        'Active_End_Frame': end_frame
    })

# --- 3. 메인 실행부 (Loop 방식) ---

# [경로 설정]
drive_root = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/dataset/Hundred'
norm_folder_path = os.path.join(drive_root, 'keypoint_Hundred', 'norm')
all_files = glob.glob(os.path.join(norm_folder_path, "norm_*.csv"))

output_path = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/analysis'

if not all_files:
    print(f"❌ '{norm_folder_path}' 경로에 'norm_*.csv' 파일이 없습니다. 경로를 확인해주세요.")
else:
    print(f"✅ 총 {len(all_files)}개의 정규화된 CSV 파일을 분석합니다.")

    results_list = []

    for i, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        print(f"  [{i+1}/{len(all_files)}] 분석 중: {filename}")

        try:
            # 1. 파일명에서 메타데이터(ID, Level) 추출 (안전한 방식으로 수정)
            clean_name = filename.replace('norm_', '').replace('.csv', '')
            parts = clean_name.split('_')

            place = parts[2]
            level = parts[5]
            person_id = parts[6]
            pose_name = parts[4]

            # 키워드로 검색 (순서가 바뀌어도 OK)
            for part in parts:
                if part.startswith('actor'):
                    person_id = part
                if part in ['고급', '중급', '초급']:
                    level = part
                if part.lower() == 'hundred':
                    pose_name = 'Hundred'

            # 'Hundred' 파일이 아니면 건너뛰기
            if pose_name != 'Hundred':
                print(f"    ... [Skip] 'Hundred' 파일이 아니므로 건너뜁니다.")
                continue

            # 2. CSV 읽기
            df_single_file = pd.read_csv(file_path)

            # 3. '안정성' 분석 (보간 + 지표 계산)
            # (calculate_hundred_metrics는 analyze_stability 내부에서 호출됨)
            stability_summary = analyze_stability(df_single_file)

            # 4. 결과 저장
            if stability_summary is not None:
                stability_summary['Source_File'] = filename
                stability_summary['Person_ID'] = person_id
                stability_summary['Level'] = level
                stability_summary['Place'] = place
                results_list.append(stability_summary)

        except KeyError as e:
            print(f"    ⚠️ {filename} 처리 중 오류 발생 (KeyError): {e} - CSV에 필요한 컬럼이 없습니다.")
        except Exception as e:
            print(f"    ⚠️ {filename} 처리 중 알 수 없는 오류 발생: {e}")

    # --- 4. 최종 결과 저장 (오류 수정된 버전) ---
    if results_list:
        df_final_eda = pd.DataFrame(results_list)

        output_dir = os.path.join(output_path, "keypoint_Hundred_Analysis_vFinal") # 새 폴더 이름
        save_path = os.path.join(output_dir, "EDA_Hundred_Stability_Report_vFinal_Longest_Streak.csv")

        os.makedirs(output_dir, exist_ok=True)
        df_final_eda.to_csv(save_path, index=False)

        print("-" * 50)
        print("✅ 'Hundred' 안정성 분석 완료! (모든 기능 통합)")
        print(f"저장된 파일: {save_path}")
        print(f"총 {len(df_final_eda)}개의 'Hundred' 영상이 분석되었습니다.")

        print("\n[결과 미리보기]")
        print(df_final_eda.head())
    else:
        print("분석할 'Hundred' 영상이 없거나 유효한 데이터가 없습니다.")

✅ 총 117개의 정규화된 CSV 파일을 분석합니다.
  [1/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP082_20221006_15.51.54_CAM_1.csv
  [2/117] 분석 중: norm_필라테스_가산_C_Mat_Hundred_고급_actorP079_20221028_14.16.15_CAM_1.csv
  [3/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.14.40_CAM_1.csv
  [4/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.15.35_CAM_1.csv
  [5/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.17.02_CAM_1.csv
  [6/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP076_20220929_15.26.04_CAM_1.csv
  [7/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.10.42_CAM_1.csv
  [8/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.11.30_CAM_1.csv
  [9/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.12.19_CAM_1.csv
  [10/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP079_20220928_12.01.17_CAM_1.csv
  [11/117] 분석 중: no

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from scipy.stats import linregress
from google.colab import drive

# 1. 드라이브 마운트 (최초 1회 실행)
try:
    drive.mount('/content/drive')
except:
    pass # 이미 마운트됨

# --- [신규 함수] 각도 계산 헬퍼 함수 ---
def get_relative_lift_angle(neck, hip, ankle):
    """
    Hip을 중심점으로, Neck-Hip 벡터와 Hip-Ankle 벡터 사이의 각도를 계산합니다.
    결과는 180 - 사이각으로, 바닥(몸통 연장선)에서 들어올린 각도를 나타냅니다.
    """
    vec_torso = neck - hip
    vec_leg = ankle - hip

    dot_product = np.dot(vec_torso, vec_leg)
    norm_torso = np.linalg.norm(vec_torso)
    norm_leg = np.linalg.norm(vec_leg)

    if norm_torso == 0 or norm_leg == 0:
        return 0

    # 코사인 법칙 계산 및 클리핑
    cosine_angle = dot_product / (norm_torso * norm_leg)
    angle_rad = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    angle_deg = np.degrees(angle_rad)

    # Hundred Lift Angle = 180도 (일직선) - 계산된 사이각
    return 180 - angle_deg

# --- 2. 분석 함수 정의 ---
def calculate_hundred_metrics(df):
    """
    Hundred 지표 (각도, 높이)를 계산
    [수정] 다리 각도 계산을 '몸통 기준'으로 변경 (Relative Angle)
    """
    df_out = df.copy()

    # (1) 다리 선택 (잘 보이는 쪽으로)
    if 'Left_Ankle_conf' not in df.columns or 'Right_Ankle_conf' not in df.columns:
        use_left_leg_mask = True
    else:
        use_left_leg_mask = df['Left_Ankle_conf'] >= df['Right_Ankle_conf']

    # 스마트 좌표 추출
    smart_ankle_x = np.where(use_left_leg_mask, df['norm_Left_Ankle_x'], df['norm_Right_Ankle_x'])
    smart_ankle_y = np.where(use_left_leg_mask, df['norm_Left_Ankle_y'], df['norm_Right_Ankle_y'])
    smart_hip_x = np.where(use_left_leg_mask, df['norm_Left_Hip_x'], df['norm_Right_Hip_x'])
    smart_hip_y = np.where(use_left_leg_mask, df['norm_Left_Hip_y'], df['norm_Right_Hip_y'])

    # (1_A) [핵심 수정] 다리 높이 각도 (몸통 기준)

    # 1. Neck Center 좌표 생성 (Vector A의 기준)
    shoulder_center_x = (df['norm_Left_Shoulder_x'] + df['norm_Right_Shoulder_x']) / 2
    shoulder_center_y = (df['norm_Left_Shoulder_y'] + df['norm_Right_Shoulder_y']) / 2

    neck_coords = np.array([shoulder_center_x, shoulder_center_y]).T # (N, 2)

    # 2. 스마트 Hip/Ankle 좌표 배열
    smart_hip_coords = np.array([smart_hip_x, smart_hip_y]).T
    smart_ankle_coords = np.array([smart_ankle_x, smart_ankle_y]).T

    # 3. Row-wise로 각도 계산 및 리스트화 (Dot Product는 벡터화가 어려움)
    angle_list = []
    for i in range(len(df)):
        hip = smart_hip_coords[i]
        ankle = smart_ankle_coords[i]
        neck = neck_coords[i]

        angle_val = get_relative_lift_angle(neck, hip, ankle)
        angle_list.append(angle_val)

    df_out['Leg_Height_Angle'] = angle_list # 새로운 각도 할당

    # (2) 코어 고정각 (Shoulder-Hip-Knee) - 이 부분은 원리상 그대로 사용
    def get_angle(a, b, c):
        ba = a - b
        bc = c - b
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

    p_shoulder = df[['norm_Left_Shoulder_x', 'norm_Left_Shoulder_y']].values
    p_hip = df[['norm_Left_Hip_x', 'norm_Left_Hip_y']].values
    p_knee = df[['norm_Left_Knee_x', 'norm_Left_Knee_y']].values

    # 코어 각도 계산은 원래 3점 사이의 각도이므로 기존 방식 유지
    df_out['Core_Angle'] = [get_angle(s, h, k) if not (np.isnan(s).any() or np.isnan(h).any() or np.isnan(k).any()) else np.nan
                            for s, h, k in zip(p_shoulder, p_hip, p_knee)]

    # (3) 상체 컬업 (Nose가 Shoulder Center보다 얼마나 위에 있나) - 유지
    shoulder_center_y = (df['norm_Left_Shoulder_y'] + df['norm_Right_Shoulder_y']) / 2
    df_out['Head_Curl_Position'] = shoulder_center_y - df['norm_Nose_y']

    return df_out

def analyze_stability(group):
    # (이하 analyze_stability 함수와 메인 실행부는 변경하지 않고 그대로 유지됩니다.)
    # ... (생략) ...

    # --- [보간(Interpolation) 전처리] ---
    parts_to_clean = ['Left_Ankle', 'Right_Ankle', 'Left_Hip', 'Right_Hip',
                      'Left_Shoulder', 'Right_Shoulder', 'Nose', 'Left_Knee']
    group_clean = group.copy()
    for part in parts_to_clean:
        if f'{part}_conf' in group_clean.columns:
            low_conf_mask = group_clean[f'{part}_conf'] < 0.5
            group_clean.loc[low_conf_mask, f'norm_{part}_x'] = np.nan
            group_clean.loc[low_conf_mask, f'norm_{part}_y'] = np.nan
    norm_cols = [col for col in group_clean.columns if col.startswith('norm_')]
    group_clean[norm_cols] = group_clean[norm_cols].interpolate(method='linear', limit_direction='both', limit=5)

    # 1. 'Hundred' 지표 계산
    group_with_metrics = calculate_hundred_metrics(group_clean)
    group_with_metrics = group_with_metrics.dropna(subset=['Leg_Height_Angle', 'Head_Curl_Position'])

    if group_with_metrics.empty:
        return None

    # --- [Longest Streak 로직] ---
    group_with_metrics['is_candidate'] = (
        (group_with_metrics['Leg_Height_Angle'] > 5) &
        (group_with_metrics['Leg_Height_Angle'] < 45) &
        (group_with_metrics['Head_Curl_Position'] > 0.05)
    )

    group_with_metrics['block_id'] = (group_with_metrics['is_candidate'] != group_with_metrics['is_candidate'].shift()).cumsum()
    candidate_streaks = group_with_metrics[group_with_metrics['is_candidate'] == True].groupby('block_id').size()

    if candidate_streaks.empty:
        return None

    longest_block_id = candidate_streaks.idxmax()
    active_frames = group_with_metrics[
        (group_with_metrics['block_id'] == longest_block_id) &
        (group_with_metrics['is_candidate'] == True)
    ].copy()
    # --- [Longest Streak 로직 끝] ---

    if len(active_frames) < 10:
        return None

    stability_score = active_frames['Leg_Height_Angle'].std()
    core_stability = active_frames['Core_Angle'].std()

    if len(active_frames['Frame'].unique()) > 1:
        slope, _ = np.polyfit(active_frames['Frame'], active_frames['Leg_Height_Angle'], 1)
    else:
        slope = 0

    start_frame = active_frames['Frame'].min()
    end_frame = active_frames['Frame'].max()

    return pd.Series({
        'Leg_Stability_StdDev': stability_score,
        'Core_Stability_StdDev': core_stability,
        'Leg_Fatigue_Slope': slope,
        'Avg_Leg_Angle': active_frames['Leg_Height_Angle'].mean(),
        'Avg_Core_Angle': active_frames['Core_Angle'].mean(),
        'Active_Frame_Count': len(active_frames),
        'Active_Start_Frame': start_frame,
        'Active_End_Frame': end_frame
    })


# --- 3. 메인 실행부 (Loop 방식) ---

# [경로 설정]
drive_root = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/dataset/Hundred'
norm_folder_path = os.path.join(drive_root, 'keypoint_Hundred', 'norm')
all_files = glob.glob(os.path.join(norm_folder_path, "norm_*.csv"))

output_path = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/analysis'

if not all_files:
    print(f"❌ '{norm_folder_path}' 경로에 'norm_*.csv' 파일이 없습니다. 경로를 확인해주세요.")
else:
    print(f"✅ 총 {len(all_files)}개의 정규화된 CSV 파일을 분석합니다.")

    results_list = []

    for i, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        print(f"  [{i+1}/{len(all_files)}] 분석 중: {filename}")

        try:
            # 1. 파일명에서 메타데이터(ID, Level) 추출 (안전한 방식으로 수정)
            clean_name = filename.replace('norm_', '').replace('.csv', '')
            parts = clean_name.split('_')

            place = parts[2]
            level = "Unknown"
            person_id = "Unknown"
            pose_name = "Unknown"

            # 키워드로 검색 (순서가 바뀌어도 OK)
            for part in parts:
                if part.startswith('actor'):
                    person_id = part
                if part in ['고급', '중급', '초급']:
                    level = part
                if part.lower() == 'hundred':
                    pose_name = 'Hundred'

            # 'Hundred' 파일이 아니면 건너뛰기
            if pose_name != 'Hundred':
                print(f"    ... [Skip] 'Hundred' 파일이 아니므로 건너뜁니다.")
                continue

            # 2. CSV 읽기
            df_single_file = pd.read_csv(file_path)

            # 3. '안정성' 분석 (보간 + 지표 계산)
            stability_summary = analyze_stability(df_single_file)

            # 4. 결과 저장
            if stability_summary is not None:
                stability_summary['Source_File'] = filename
                stability_summary['Person_ID'] = person_id
                stability_summary['Level'] = level
                stability_summary['Place'] = place
                results_list.append(stability_summary)

        except KeyError as e:
            print(f"    ⚠️ {filename} 처리 중 오류 발생 (KeyError): {e} - CSV에 필요한 컬럼이 없습니다.")
        except Exception as e:
            print(f"    ⚠️ {filename} 처리 중 알 수 없는 오류 발생: {e}")

    # --- 4. 최종 결과 저장 (오류 수정된 버전) ---
    if results_list:
        df_final_eda = pd.DataFrame(results_list)

        output_dir = os.path.join(output_path, "keypoint_Hundred_Analysis_vFinal") # 새 폴더 이름
        save_path = os.path.join(output_dir, "EDA_Hundred_Stability_Report_vFinal_BodyRelative.csv") # 파일명 변경

        os.makedirs(output_dir, exist_ok=True)
        df_final_eda.to_csv(save_path, index=False)

        print("-" * 50)
        print("✅ 'Hundred' 안정성 분석 완료! (몸통 기준 각도 적용)")
        print(f"저장된 파일: {save_path}")
        print(f"총 {len(df_final_eda)}개의 'Hundred' 영상이 분석되었습니다.")

        print("\n[결과 미리보기]")
        print(df_final_eda.head())
    else:
        print("분석할 'Hundred' 영상이 없거나 유효한 데이터가 없습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 총 117개의 정규화된 CSV 파일을 분석합니다.
  [1/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP082_20221006_15.51.54_CAM_1.csv
  [2/117] 분석 중: norm_필라테스_가산_C_Mat_Hundred_고급_actorP079_20221028_14.16.15_CAM_1.csv
  [3/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.14.40_CAM_1.csv
  [4/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.15.35_CAM_1.csv
  [5/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.17.02_CAM_1.csv
  [6/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP076_20220929_15.26.04_CAM_1.csv
  [7/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.10.42_CAM_1.csv
  [8/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.11.30_CAM_1.csv
  [9/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.12

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from scipy.stats import linregress
from sklearn.cluster import KMeans # [추가] 군집화를 위한 라이브러리
from google.colab import drive

# 1. 드라이브 마운트
try:
    drive.mount('/content/drive')
except:
    pass

# --- 2. 분석 함수 정의 ---

def calculate_hundred_metrics(df):
    """
    [유지] Hundred 지표 (각도, 높이) 계산 함수
    """
    df_out = df.copy()

    # (1) 다리 선택 (잘 보이는 쪽으로)
    if 'Left_Ankle_conf' not in df.columns or 'Right_Ankle_conf' not in df.columns:
        use_left_leg_mask = True
    else:
        use_left_leg_mask = df['Left_Ankle_conf'] >= df['Right_Ankle_conf']

    smart_ankle_x = np.where(use_left_leg_mask, df['norm_Left_Ankle_x'], df['norm_Right_Ankle_x'])
    smart_ankle_y = np.where(use_left_leg_mask, df['norm_Left_Ankle_y'], df['norm_Right_Ankle_y'])
    smart_hip_x = np.where(use_left_leg_mask, df['norm_Left_Hip_x'], df['norm_Right_Hip_x'])
    smart_hip_y = np.where(use_left_leg_mask, df['norm_Left_Hip_y'], df['norm_Right_Hip_y'])

    # (1_A) 다리 높이 각도 (Smart Leg 기준)
    dy = -(smart_ankle_y - smart_hip_y)
    dx = np.abs(smart_ankle_x - smart_hip_x)
    df_out['Leg_Height_Angle'] = np.degrees(np.arctan2(dy, dx))

    # (2) 코어 고정각
    def get_angle(a, b, c):
        ba = a - b
        bc = c - b
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

    p_shoulder = df[['norm_Left_Shoulder_x', 'norm_Left_Shoulder_y']].values
    p_hip = df[['norm_Left_Hip_x', 'norm_Left_Hip_y']].values
    p_knee = df[['norm_Left_Knee_x', 'norm_Left_Knee_y']].values

    df_out['Core_Angle'] = [get_angle(s, h, k) if not (np.isnan(s).any() or np.isnan(h).any() or np.isnan(k).any()) else np.nan
                            for s, h, k in zip(p_shoulder, p_hip, p_knee)]

    # (3) 상체 컬업
    shoulder_center_y = (df['norm_Left_Shoulder_y'] + df['norm_Right_Shoulder_y']) / 2
    df_out['Head_Curl_Position'] = shoulder_center_y - df['norm_Nose_y']

    return df_out

def analyze_stability(group):
    """
    [전면 수정] K-Means 군집화를 이용해 'Active Frame' 자동 감지
    """

    # --- [1. 보간(Interpolation) 전처리] ---
    parts_to_clean = ['Left_Ankle', 'Right_Ankle', 'Left_Hip', 'Right_Hip',
                      'Left_Shoulder', 'Right_Shoulder', 'Nose', 'Left_Knee']
    group_clean = group.copy()
    for part in parts_to_clean:
        if f'{part}_conf' in group_clean.columns:
            low_conf_mask = group_clean[f'{part}_conf'] < 0.5
            group_clean.loc[low_conf_mask, f'norm_{part}_x'] = np.nan
            group_clean.loc[low_conf_mask, f'norm_{part}_y'] = np.nan

    norm_cols = [col for col in group_clean.columns if col.startswith('norm_')]
    group_clean[norm_cols] = group_clean[norm_cols].interpolate(method='linear', limit_direction='both', limit=5)

    # --- [2. 지표 계산] ---
    group_with_metrics = calculate_hundred_metrics(group_clean)

    # 분석을 위해 NaN이 있는 행은 잠시 제외 (K-Means 오류 방지)
    valid_data = group_with_metrics.dropna(subset=['Leg_Height_Angle', 'Head_Curl_Position']).copy()

    if len(valid_data) < 30: # 데이터가 너무 적으면 분석 불가
        return None

    # --- [3. K-Means 군집화로 Active 구간 찾기] ---
    # 데이터를 2개 그룹(휴식 vs 운동)으로 나눔
    X = valid_data[['Leg_Height_Angle']].values

    try:
        # k=2로 군집화 (Random State 고정으로 재현성 확보)
        kmeans = KMeans(n_clusters=2, n_init=10, random_state=42)
        valid_data['cluster'] = kmeans.fit_predict(X)

        # 각 군집의 평균 각도 계산
        cluster_means = valid_data.groupby('cluster')['Leg_Height_Angle'].mean()

        # 평균 각도가 더 높은 군집을 'Active' 군집으로 선정
        active_cluster_id = cluster_means.idxmax()
        active_mean_angle = cluster_means.max()

        # [최소 조건] 아무리 높아도 평균 5도 미만이면 운동 안 한 것으로 간주
        if active_mean_angle < 5:
            return None

        # Active 군집인 프레임에만 True 표시
        valid_data['is_candidate'] = valid_data['cluster'] == active_cluster_id

    except Exception as e:
        print(f"K-Means Error: {e}")
        return None

    # --- [4. 가장 긴 연속 구간(Longest Streak) 추출] ---
    # 군집화 결과에서 노이즈를 제거하고 가장 길게 유지한 구간만 선택

    # 연속된 그룹 ID 부여
    valid_data['block_id'] = (valid_data['is_candidate'] != valid_data['is_candidate'].shift()).cumsum()

    # 'True(Active)'인 블록들의 길이 계산
    candidate_streaks = valid_data[valid_data['is_candidate'] == True].groupby('block_id').size()

    if candidate_streaks.empty:
        return None

    # 가장 긴 블록 선택
    longest_block_id = candidate_streaks.idxmax()

    # 최종 Active Frames 추출
    active_frames = valid_data[
        (valid_data['block_id'] == longest_block_id) &
        (valid_data['is_candidate'] == True)
    ].copy()

    if len(active_frames) < 10:
        return None

    # --- [5. 최종 통계 산출] ---
    stability_score = active_frames['Leg_Height_Angle'].std()
    core_stability = active_frames['Core_Angle'].std()

    if len(active_frames['Frame'].unique()) > 1:
        slope, _ = np.polyfit(active_frames['Frame'], active_frames['Leg_Height_Angle'], 1)
    else:
        slope = 0

    start_frame = active_frames['Frame'].min()
    end_frame = active_frames['Frame'].max()

    return pd.Series({
        'Leg_Stability_StdDev': stability_score,
        'Core_Stability_StdDev': core_stability,
        'Leg_Fatigue_Slope': slope,
        'Avg_Leg_Angle': active_frames['Leg_Height_Angle'].mean(),
        'Avg_Core_Angle': active_frames['Core_Angle'].mean(),
        'Active_Frame_Count': len(active_frames),
        'Active_Start_Frame': start_frame,
        'Active_End_Frame': end_frame
    })

# --- 3. 메인 실행부 (Loop 방식) ---

# [경로 설정]
drive_root = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/dataset/Hundred'
norm_folder_path = os.path.join(drive_root, 'norm')
all_files = glob.glob(os.path.join(norm_folder_path, "norm_*.csv"))

output_path = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/analysis'

if not all_files:
    print(f"❌ '{norm_folder_path}' 경로에 'norm_*.csv' 파일이 없습니다.")
else:
    print(f"✅ 총 {len(all_files)}개의 파일을 분석합니다 (K-Means 적용).")

    results_list = []

    for i, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        # 진행 상황 출력 (너무 많으면 줄이셔도 됩니다)
        if i % 10 == 0:
            print(f"  [{i+1}/{len(all_files)}] 처리 중...")

        try:
            # 메타데이터 추출
            clean_name = filename.replace('norm_', '').replace('.csv', '')
            parts = clean_name.split('_')

            place = parts[2]
            level = parts[5]
            person_id = parts[6]
            pose_name = parts[4]

            for part in parts:
                if part.startswith('actor'): person_id = part
                if part in ['고급', '중급', '초급']: level = part
                if part.lower() == 'hundred': pose_name = 'Hundred'

            if pose_name != 'Hundred':
                continue

            # 데이터 읽기 및 분석
            df_single_file = pd.read_csv(file_path)
            stability_summary = analyze_stability(df_single_file)

            if stability_summary is not None:
                stability_summary['Source_File'] = filename
                stability_summary['Person_ID'] = person_id
                stability_summary['Level'] = level
                stability_summary['Place'] = place
                results_list.append(stability_summary)

        except Exception as e:
            print(f"    ⚠️ {filename} 오류: {e}")

    # 저장
    if results_list:
        df_final_eda = pd.DataFrame(results_list)

        output_dir = os.path.join(output_path, "keypoint_Hundred_Analysis_KMeans")
        save_path = os.path.join(output_dir, "EDA_Hundred_Stability_Report_KMeans.csv")

        os.makedirs(output_dir, exist_ok=True)
        df_final_eda.to_csv(save_path, index=False)

        print("-" * 50)
        print("✅ 'Hundred' K-Means 기반 분석 완료!")
        print(f"저장된 파일: {save_path}")
        print(df_final_eda.head())
    else:
        print("분석 결과가 없습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 총 117개의 파일을 분석합니다 (K-Means 적용).
  [1/117] 처리 중...
  [11/117] 처리 중...
  [21/117] 처리 중...
  [31/117] 처리 중...
  [41/117] 처리 중...
  [51/117] 처리 중...
  [61/117] 처리 중...
  [71/117] 처리 중...
  [81/117] 처리 중...
  [91/117] 처리 중...
  [101/117] 처리 중...
  [111/117] 처리 중...
--------------------------------------------------
✅ 'Hundred' K-Means 기반 분석 완료!
저장된 파일: /content/drive/MyDrive/shared_googledrive(Sessac Final Project)/analysis/keypoint_Hundred_Analysis_KMeans/EDA_Hundred_Stability_Report_KMeans.csv
   Leg_Stability_StdDev  Core_Stability_StdDev  Leg_Fatigue_Slope  \
0             13.858505              27.198207           0.014282   
1             10.132320              30.215632           0.021984   

   Avg_Leg_Angle  Avg_Core_Angle  Active_Frame_Count  Active_Start_Frame  \
0       5.434976      153.699158               386.0               134.0   
1       5.009

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from scipy.stats import linregress
from google.colab import drive

# 1. 드라이브 마운트 (최초 1회 실행)

# --- 2. 분석 함수 정의 ---
def calculate_hundred_metrics(df):
    """
    Hundred 지표 (각도, 높이)를 계산
    - (Smart Leg) 신뢰도 높은 다리 자동 선택
    - (abs(dx)) 방향에 상관없이 예각(Angle) 계산
    """
    df_out = df.copy()

    # (1) 다리 선택 (잘 보이는 쪽으로)
    # 원본 _conf 컬럼이 있는지 확인
    if 'Left_Ankle_conf' not in df.columns or 'Right_Ankle_conf' not in df.columns:
        print("    ⚠️ Ankle_conf 컬럼이 없어 '왼쪽 다리'를 기준으로 계산합니다.")
        use_left_leg_mask = True # conf 없으면 왼쪽 기준
    else:
        use_left_leg_mask = df['Left_Ankle_conf'] >= df['Right_Ankle_conf']

    smart_ankle_x = np.where(use_left_leg_mask, df['norm_Left_Ankle_x'], df['norm_Right_Ankle_x'])
    smart_ankle_y = np.where(use_left_leg_mask, df['norm_Left_Ankle_y'], df['norm_Right_Ankle_y'])
    smart_hip_x = np.where(use_left_leg_mask, df['norm_Left_Hip_x'], df['norm_Right_Hip_x'])
    smart_hip_y = np.where(use_left_leg_mask, df['norm_Left_Hip_y'], df['norm_Right_Hip_y'])

    # (1_A) 다리 높이 각도 (Smart Leg 기준)
    dy = -(smart_ankle_y - smart_hip_y) # Y축 반전
    dx = np.abs(smart_ankle_x - smart_hip_x) # [수정] 절댓값 적용
    df_out['Leg_Height_Angle'] = np.degrees(np.arctan2(dy, dx))

    # (2) 코어 고정각 (Shoulder-Hip-Knee)
    def get_angle(a, b, c):
        ba = a - b
        bc = c - b
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

    p_shoulder = df[['norm_Left_Shoulder_x', 'norm_Left_Shoulder_y']].values
    p_hip = df[['norm_Left_Hip_x', 'norm_Left_Hip_y']].values
    p_knee = df[['norm_Left_Knee_x', 'norm_Left_Knee_y']].values

    df_out['Core_Angle'] = [get_angle(s, h, k) if not (np.isnan(s).any() or np.isnan(h).any() or np.isnan(k).any()) else np.nan
                            for s, h, k in zip(p_shoulder, p_hip, p_knee)]

    # (3) 상체 컬업 (Nose가 Shoulder Center보다 얼마나 위에 있나)
    shoulder_center_y = (df['norm_Left_Shoulder_y'] + df['norm_Right_Shoulder_y']) / 2
    df_out['Head_Curl_Position'] = shoulder_center_y - df['norm_Nose_y']

    return df_out

def analyze_stability(group):
    """
    [심플 버전] 'Hundred'는 누워서 하는 운동이다!
    -> 코(Nose)가 '누워있는 높이'에 있을 때만 잘라내서 분석
    """

    # --- [1. 보간(Interpolation) 전처리] ---
    # (데이터 끊김 방지용 필수 코드는 유지)
    parts_to_clean = ['Left_Ankle', 'Right_Ankle', 'Left_Hip', 'Right_Hip',
                      'Left_Shoulder', 'Right_Shoulder', 'Nose', 'Left_Knee']
    group_clean = group.copy()
    for part in parts_to_clean:
        if f'{part}_conf' in group_clean.columns:
            low_conf_mask = group_clean[f'{part}_conf'] < 0.5
            group_clean.loc[low_conf_mask, f'norm_{part}_x'] = np.nan
            group_clean.loc[low_conf_mask, f'norm_{part}_y'] = np.nan
    norm_cols = [col for col in group_clean.columns if col.startswith('norm_')]
    group_clean[norm_cols] = group_clean[norm_cols].interpolate(method='linear', limit_direction='both', limit=5)

    # 1. 지표 계산
    group_with_metrics = calculate_hundred_metrics(group_clean)

    # --- [2. 핵심 수정] Nose_Y 기반 단순 컷팅 ---

    # (1) 코의 높이 범위 계산
    min_nose_y = group_with_metrics['norm_Nose_y'].min() # 제일 높을 때 (앉음)
    max_nose_y = group_with_metrics['norm_Nose_y'].max() # 제일 낮을 때 (누움)

    # (2) "누움"의 기준선(Threshold) 정하기
    # "전체 높이 변화의 하위 40% 지점보다 더 아래(Max쪽에 가까움)에 있으면 누운 것이다"
    # (이미지 좌표계는 아래로 갈수록 값이 커지므로 max에 가까워야 누운 것)
    threshold = max_nose_y - (max_nose_y - min_nose_y) * 0.4

    # (3) 기준선보다 아래(누워있음)인 프레임만 선택
    active_frames = group_with_metrics[
        group_with_metrics['norm_Nose_y'] > threshold
    ].copy()

    # (4) 노이즈 제거: 너무 짧은 구간(1초 미만)은 무시하고 가장 긴 덩어리만 선택
    # (앉았다가 잠깐 눕는 척 하거나 튀는 값 방지)
    if active_frames.empty: return None

    # 연속 구간 ID 부여
    active_frames['frame_diff'] = active_frames['Frame'].diff()
    active_frames['block_id'] = (active_frames['frame_diff'] > 5).cumsum() # 5프레임 이상 끊기면 다른 블록

    # 가장 긴 블록 선택
    longest_block_id = active_frames.groupby('block_id').size().idxmax()
    final_active_frames = active_frames[active_frames['block_id'] == longest_block_id]

    if len(final_active_frames) < 30: # 1초(30프레임)도 안 누워있으면 무시
        return None

    # --- [3. 최종 통계 산출] ---
    stability_score = final_active_frames['Leg_Height_Angle'].std()
    core_stability = final_active_frames['Core_Angle'].std()

    if len(final_active_frames['Frame'].unique()) > 1:
        slope, _ = np.polyfit(final_active_frames['Frame'], final_active_frames['Leg_Height_Angle'], 1)
    else:
        slope = 0

    start_frame = final_active_frames['Frame'].min()
    end_frame = final_active_frames['Frame'].max()

    return pd.Series({
        'Leg_Stability_StdDev': stability_score,
        'Core_Stability_StdDev': core_stability,
        'Leg_Fatigue_Slope': slope,
        'Avg_Leg_Angle': final_active_frames['Leg_Height_Angle'].mean(),
        'Avg_Core_Angle': final_active_frames['Core_Angle'].mean(),
        'Active_Frame_Count': len(final_active_frames),
        'Active_Start_Frame': start_frame,
        'Active_End_Frame': end_frame
    })

# --- 3. 메인 실행부 (Loop 방식) ---

# [경로 설정]
drive_root = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/dataset/Hundred'
norm_folder_path = os.path.join(drive_root, 'norm')
all_files = glob.glob(os.path.join(norm_folder_path, "norm_*.csv"))

output_path = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/analysis'

if not all_files:
    print(f"❌ '{norm_folder_path}' 경로에 'norm_*.csv' 파일이 없습니다. 경로를 확인해주세요.")
else:
    print(f"✅ 총 {len(all_files)}개의 정규화된 CSV 파일을 분석합니다.")

    results_list = []

    for i, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        print(f"  [{i+1}/{len(all_files)}] 분석 중: {filename}")

        try:
            # 1. 파일명에서 메타데이터(ID, Level) 추출 (안전한 방식으로 수정)
            clean_name = filename.replace('norm_', '').replace('.csv', '')
            parts = clean_name.split('_')

            place = parts[2]
            level = parts[5]
            person_id = parts[6]
            pose_name = parts[4]

            # 키워드로 검색 (순서가 바뀌어도 OK)
            for part in parts:
                if part.startswith('actor'):
                    person_id = part
                if part in ['고급', '중급', '초급']:
                    level = part
                if part.lower() == 'hundred':
                    pose_name = 'Hundred'

            # 'Hundred' 파일이 아니면 건너뛰기
            if pose_name != 'Hundred':
                print(f"    ... [Skip] 'Hundred' 파일이 아니므로 건너뜁니다.")
                continue

            # 2. CSV 읽기
            df_single_file = pd.read_csv(file_path)

            # 3. '안정성' 분석 (보간 + 지표 계산)
            # (calculate_hundred_metrics는 analyze_stability 내부에서 호출됨)
            stability_summary = analyze_stability(df_single_file)

            # 4. 결과 저장
            if stability_summary is not None:
                stability_summary['Source_File'] = filename
                stability_summary['Person_ID'] = person_id
                stability_summary['Level'] = level
                stability_summary['Place'] = place
                results_list.append(stability_summary)

        except KeyError as e:
            print(f"    ⚠️ {filename} 처리 중 오류 발생 (KeyError): {e} - CSV에 필요한 컬럼이 없습니다.")
        except Exception as e:
            print(f"    ⚠️ {filename} 처리 중 알 수 없는 오류 발생: {e}")

    # --- 4. 최종 결과 저장 (오류 수정된 버전) ---
    if results_list:
        df_final_eda = pd.DataFrame(results_list)

        output_dir = os.path.join(output_path, "keypoint_Hundred_Analysis_noseY_active") # 새 폴더 이름
        save_path = os.path.join(output_dir, "EDA_Hundred_Stability_Report_noseY_1119.csv")

        os.makedirs(output_dir, exist_ok=True)
        df_final_eda.to_csv(save_path, index=False)

        print("-" * 50)
        print("✅ 'Hundred' 안정성 분석 완료! (모든 기능 통합)")
        print(f"저장된 파일: {save_path}")
        print(f"총 {len(df_final_eda)}개의 'Hundred' 영상이 분석되었습니다.")

        print("\n[결과 미리보기]")
        print(df_final_eda.head())
    else:
        print("분석할 'Hundred' 영상이 없거나 유효한 데이터가 없습니다.")

✅ 총 117개의 정규화된 CSV 파일을 분석합니다.
  [1/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.14.40_CAM_1.csv
  [2/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.15.35_CAM_1.csv
  [3/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.17.02_CAM_1.csv
  [4/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP076_20220929_15.26.04_CAM_1.csv
  [5/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.10.42_CAM_1.csv
  [6/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.11.30_CAM_1.csv
  [7/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.12.19_CAM_1.csv
  [8/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP079_20220928_12.01.17_CAM_1.csv
  [9/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP079_20220928_12.02.15_CAM_1.csv
  [10/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP079_20220928_12.03.10_CAM_1.csv
  [11/117] 분석 중: no

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from scipy.stats import linregress
from google.colab import drive

# 1. 드라이브 마운트 (최초 1회 실행)

# --- 2. 분석 함수 정의 ---
def calculate_hundred_metrics(df):
    """
    Hundred 지표 (각도, 높이)를 계산
    - (Smart Leg) 신뢰도 높은 다리 자동 선택
    - (abs(dx)) 방향에 상관없이 예각(Angle) 계산
    """
    df_out = df.copy()

    # (1) 다리 선택 (잘 보이는 쪽으로)
    # 원본 _conf 컬럼이 있는지 확인
    if 'Left_Ankle_conf' not in df.columns or 'Right_Ankle_conf' not in df.columns:
        print("    ⚠️ Ankle_conf 컬럼이 없어 '왼쪽 다리'를 기준으로 계산합니다.")
        use_left_leg_mask = True # conf 없으면 왼쪽 기준
    else:
        use_left_leg_mask = df['Left_Ankle_conf'] >= df['Right_Ankle_conf']

    smart_ankle_x = np.where(use_left_leg_mask, df['norm_Left_Ankle_x'], df['norm_Right_Ankle_x'])
    smart_ankle_y = np.where(use_left_leg_mask, df['norm_Left_Ankle_y'], df['norm_Right_Ankle_y'])
    smart_hip_x = np.where(use_left_leg_mask, df['norm_Left_Hip_x'], df['norm_Right_Hip_x'])
    smart_hip_y = np.where(use_left_leg_mask, df['norm_Left_Hip_y'], df['norm_Right_Hip_y'])

    # (1_A) 다리 높이 각도 (Smart Leg 기준)
    dy = -(smart_ankle_y - smart_hip_y) # Y축 반전
    dx = np.abs(smart_ankle_x - smart_hip_x) # [수정] 절댓값 적용
    df_out['Leg_Height_Angle'] = np.degrees(np.arctan2(dy, dx))

    # (2) 코어 고정각 (Shoulder-Hip-Knee)
    def get_angle(a, b, c):
        ba = a - b
        bc = c - b
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

    p_shoulder = df[['norm_Left_Shoulder_x', 'norm_Left_Shoulder_y']].values
    p_hip = df[['norm_Left_Hip_x', 'norm_Left_Hip_y']].values
    p_knee = df[['norm_Left_Knee_x', 'norm_Left_Knee_y']].values

    df_out['Core_Angle'] = [get_angle(s, h, k) if not (np.isnan(s).any() or np.isnan(h).any() or np.isnan(k).any()) else np.nan
                            for s, h, k in zip(p_shoulder, p_hip, p_knee)]

    # (3) 상체 컬업 (Nose가 Shoulder Center보다 얼마나 위에 있나)
    shoulder_center_y = (df['norm_Left_Shoulder_y'] + df['norm_Right_Shoulder_y']) / 2
    df_out['Head_Curl_Position'] = shoulder_center_y - df['norm_Nose_y']

    return df_out

def get_dominant_angle(series, bin_size=1.0):
    """
    연속된 각도 데이터에서 '가장 오랫동안 유지된 각도(최빈값)'를 찾습니다.
    bin_size: 각도를 뭉뚱그릴 단위 (1.0도 단위로 반올림해서 셈)
    """
    if len(series) == 0: return np.nan

    # 1. 각도를 정수(또는 bin_size) 단위로 반올림 (노이즈 제거)
    rounded_series = (series / bin_size).round() * bin_size

    # 2. 가장 많이 등장한 값(Mode) 찾기
    mode_value = rounded_series.mode()

    if len(mode_value) > 0:
        return mode_value[0] # 최빈값이 여러 개면 첫 번째 것 선택
    return np.nan

def analyze_stability(group):
    """
    [최신 수정]
    1. Nose_Y로 Active 구간 자르기
    2. '평균' 대신 '가장 오래 유지한 각도(Dominant Angle)' 추출
    """

    # --- [1. 보간(Interpolation) 전처리] ---
    parts_to_clean = ['Left_Ankle', 'Right_Ankle', 'Left_Hip', 'Right_Hip',
                      'Left_Shoulder', 'Right_Shoulder', 'Nose', 'Left_Knee']
    group_clean = group.copy()
    for part in parts_to_clean:
        if f'{part}_conf' in group_clean.columns:
            low_conf_mask = group_clean[f'{part}_conf'] < 0.5
            group_clean.loc[low_conf_mask, f'norm_{part}_x'] = np.nan
            group_clean.loc[low_conf_mask, f'norm_{part}_y'] = np.nan
    norm_cols = [col for col in group_clean.columns if col.startswith('norm_')]
    group_clean[norm_cols] = group_clean[norm_cols].interpolate(method='linear', limit_direction='both', limit=5)

    # 1. 지표 계산 (몸통 기준 각도 함수 사용)
    group_with_metrics = calculate_hundred_metrics(group_clean)

    # --- [2. Active 구간 자르기 (Nose_Y 로직)] ---
    min_nose_y = group_with_metrics['norm_Nose_y'].min()
    max_nose_y = group_with_metrics['norm_Nose_y'].max()
    threshold = max_nose_y - (max_nose_y - min_nose_y) * 0.4 # 하위 40%

    active_frames = group_with_metrics[group_with_metrics['norm_Nose_y'] > threshold].copy()

    if active_frames.empty: return None

    # 가장 긴 연속 구간 선택
    active_frames['frame_diff'] = active_frames['Frame'].diff()
    active_frames['block_id'] = (active_frames['frame_diff'] > 5).cumsum()
    longest_block_id = active_frames.groupby('block_id').size().idxmax()
    final_active_frames = active_frames[active_frames['block_id'] == longest_block_id]

    if len(final_active_frames) < 30: return None # 1초 미만 무시

    # --- [3. 핵심 수정: Dominant Angle & 유지율 계산] ---

    # (1) Dominant Angle (가장 오래 유지한 각도)
    dominant_leg_angle = get_dominant_angle(final_active_frames['Leg_Height_Angle'])
    dominant_core_angle = get_dominant_angle(final_active_frames['Core_Angle'])

    # (2) 유지율 (Consistency Score)
    # Dominant Angle에서 ±5도 이내에 들어온 프레임 비율
    leg_error_margin = 5.0
    frames_in_zone = final_active_frames[
        (final_active_frames['Leg_Height_Angle'] >= dominant_leg_angle - leg_error_margin) &
        (final_active_frames['Leg_Height_Angle'] <= dominant_leg_angle + leg_error_margin)
    ]
    consistency_score = (len(frames_in_zone) / len(final_active_frames)) * 100 # 퍼센트(%)

    # (3) 기존 통계량
    stability_score = final_active_frames['Leg_Height_Angle'].std() # 흔들림

    if len(final_active_frames['Frame'].unique()) > 1:
        slope, _ = np.polyfit(final_active_frames['Frame'], final_active_frames['Leg_Height_Angle'], 1)
    else:
        slope = 0

    start_frame = final_active_frames['Frame'].min()
    end_frame = final_active_frames['Frame'].max()

    return pd.Series({
        'Dominant_Leg_Angle': dominant_leg_angle,   # [New] 사용자가 의도한 각도
        'Dominant_Core_Angle': dominant_core_angle, # [New] 사용자가 의도한 코어각
        'Consistency_Score': consistency_score,     # [New] 유지율 (%)
        'Leg_Stability_StdDev': stability_score,    # 흔들림
        'Leg_Fatigue_Slope': slope,                 # 처짐 기울기
        'Active_Frame_Count': len(final_active_frames),
        'Active_Start_Frame': start_frame,
        'Active_End_Frame': end_frame
    })


# --- 3. 메인 실행부 (Loop 방식) ---

# [경로 설정]
drive_root = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/dataset/Hundred'
norm_folder_path = os.path.join(drive_root, 'norm')
all_files = glob.glob(os.path.join(norm_folder_path, "norm_*.csv"))

output_path = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/analysis'

if not all_files:
    print(f"❌ '{norm_folder_path}' 경로에 'norm_*.csv' 파일이 없습니다. 경로를 확인해주세요.")
else:
    print(f"✅ 총 {len(all_files)}개의 정규화된 CSV 파일을 분석합니다.")

    results_list = []

    for i, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        print(f"  [{i+1}/{len(all_files)}] 분석 중: {filename}")

        try:
            # 1. 파일명에서 메타데이터(ID, Level) 추출 (안전한 방식으로 수정)
            clean_name = filename.replace('norm_', '').replace('.csv', '')
            parts = clean_name.split('_')

            place = parts[2]
            level = parts[5]
            person_id = parts[6]
            pose_name = parts[4]

            # 키워드로 검색 (순서가 바뀌어도 OK)
            for part in parts:
                if part.startswith('actor'):
                    person_id = part
                if part in ['고급', '중급', '초급']:
                    level = part
                if part.lower() == 'hundred':
                    pose_name = 'Hundred'

            # 'Hundred' 파일이 아니면 건너뛰기
            if pose_name != 'Hundred':
                print(f"    ... [Skip] 'Hundred' 파일이 아니므로 건너뜁니다.")
                continue

            # 2. CSV 읽기
            df_single_file = pd.read_csv(file_path)

            # 3. '안정성' 분석 (보간 + 지표 계산)
            # (calculate_hundred_metrics는 analyze_stability 내부에서 호출됨)
            stability_summary = analyze_stability(df_single_file)

            # 4. 결과 저장
            if stability_summary is not None:
                stability_summary['Source_File'] = filename
                stability_summary['Person_ID'] = person_id
                stability_summary['Level'] = level
                stability_summary['Place'] = place
                results_list.append(stability_summary)

        except KeyError as e:
            print(f"    ⚠️ {filename} 처리 중 오류 발생 (KeyError): {e} - CSV에 필요한 컬럼이 없습니다.")
        except Exception as e:
            print(f"    ⚠️ {filename} 처리 중 알 수 없는 오류 발생: {e}")

    # --- 4. 최종 결과 저장 (오류 수정된 버전) ---
    if results_list:
        df_final_eda = pd.DataFrame(results_list)

        output_dir = os.path.join(output_path, "keypoint_Hundred_Analysis") # 새 폴더 이름
        save_path = os.path.join(output_dir, "EDA_Hundred_Stability_Report_mode_1119.csv")

        os.makedirs(output_dir, exist_ok=True)
        df_final_eda.to_csv(save_path, index=False)

        print("-" * 50)
        print("✅ 'Hundred' 안정성 분석 완료! (모든 기능 통합)")
        print(f"저장된 파일: {save_path}")
        print(f"총 {len(df_final_eda)}개의 'Hundred' 영상이 분석되었습니다.")

        print("\n[결과 미리보기]")
        print(df_final_eda.head())
    else:
        print("분석할 'Hundred' 영상이 없거나 유효한 데이터가 없습니다.")

✅ 총 117개의 정규화된 CSV 파일을 분석합니다.
  [1/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.14.40_CAM_1.csv
  [2/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.15.35_CAM_1.csv
  [3/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.17.02_CAM_1.csv
  [4/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP076_20220929_15.26.04_CAM_1.csv
  [5/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.10.42_CAM_1.csv
  [6/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.11.30_CAM_1.csv
  [7/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP077_20220929_11.12.19_CAM_1.csv
  [8/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP079_20220928_12.01.17_CAM_1.csv
  [9/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP079_20220928_12.02.15_CAM_1.csv
  [10/117] 분석 중: norm_필라테스_가산_A_Mat_Hundred_고급_actorP079_20220928_12.03.10_CAM_1.csv
  [11/117] 분석 중: no

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from scipy.stats import linregress
from google.colab import drive

# 1. 드라이브 마운트
try:
    drive.mount('/content/drive')
except:
    pass

# --- [헬퍼 함수] 최빈값 계산 ---
def get_dominant_angle(series, bin_size=1.0):
    """
    가장 오랫동안 유지된 각도(최빈값)를 찾습니다.
    """
    if len(series) == 0: return np.nan
    # 각도를 정수(bin_size) 단위로 반올림하여 노이즈 제거 후 최빈값 찾기
    rounded_series = (series / bin_size).round() * bin_size
    mode_value = rounded_series.mode()

    if len(mode_value) > 0:
        return mode_value[0]
    return np.nan

# --- [헬퍼 함수] 몸통 기준 각도 계산 ---
def get_relative_lift_angle(neck, hip, ankle):
    vec_torso = neck - hip
    vec_leg = ankle - hip

    dot_product = np.dot(vec_torso, vec_leg)
    norm_torso = np.linalg.norm(vec_torso)
    norm_leg = np.linalg.norm(vec_leg)

    if norm_torso == 0 or norm_leg == 0: return 0

    cosine_angle = dot_product / (norm_torso * norm_leg)
    angle_rad = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    angle_deg = np.degrees(angle_rad)

    return 180 - angle_deg

# --- 2. 지표 계산 함수 ---
def calculate_hundred_metrics(df):
    df_out = df.copy()

    # (1) 다리 선택
    if 'Left_Ankle_conf' not in df.columns or 'Right_Ankle_conf' not in df.columns:
        use_left_leg_mask = True
    else:
        use_left_leg_mask = df['Left_Ankle_conf'] >= df['Right_Ankle_conf']

    smart_ankle_x = np.where(use_left_leg_mask, df['norm_Left_Ankle_x'], df['norm_Right_Ankle_x'])
    smart_ankle_y = np.where(use_left_leg_mask, df['norm_Left_Ankle_y'], df['norm_Right_Ankle_y'])
    smart_hip_x = np.where(use_left_leg_mask, df['norm_Left_Hip_x'], df['norm_Right_Hip_x'])
    smart_hip_y = np.where(use_left_leg_mask, df['norm_Left_Hip_y'], df['norm_Right_Hip_y'])

    # (1_A) 다리 높이 각도 (몸통 기준 Relative Angle)
    shoulder_center_x = (df['norm_Left_Shoulder_x'] + df['norm_Right_Shoulder_x']) / 2
    shoulder_center_y = (df['norm_Left_Shoulder_y'] + df['norm_Right_Shoulder_y']) / 2

    neck_coords = np.array([shoulder_center_x, shoulder_center_y]).T
    smart_hip_coords = np.array([smart_hip_x, smart_hip_y]).T
    smart_ankle_coords = np.array([smart_ankle_x, smart_ankle_y]).T

    angle_list = []
    for i in range(len(df)):
        angle_val = get_relative_lift_angle(neck_coords[i], smart_hip_coords[i], smart_ankle_coords[i])
        angle_list.append(angle_val)

    df_out['Leg_Height_Angle'] = angle_list

    # (2) 코어 고정각
    def get_angle(a, b, c):
        ba = a - b
        bc = c - b
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

    p_shoulder = df[['norm_Left_Shoulder_x', 'norm_Left_Shoulder_y']].values
    p_hip = df[['norm_Left_Hip_x', 'norm_Left_Hip_y']].values
    p_knee = df[['norm_Left_Knee_x', 'norm_Left_Knee_y']].values

    df_out['Core_Angle'] = [get_angle(s, h, k) if not (np.isnan(s).any() or np.isnan(h).any() or np.isnan(k).any()) else np.nan
                            for s, h, k in zip(p_shoulder, p_hip, p_knee)]

    # (3) 상체 컬업
    df_out['Head_Curl_Position'] = shoulder_center_y - df['norm_Nose_y']

    return df_out

def analyze_stability(group):
    """
    [수정됨] Nose_Y 로직 폐기 -> 다리 각도(>5도) 기준으로 Active 구간 선정
    """

    # --- [1. 보간(Interpolation)] ---
    parts_to_clean = ['Left_Ankle', 'Right_Ankle', 'Left_Hip', 'Right_Hip',
                      'Left_Shoulder', 'Right_Shoulder', 'Nose', 'Left_Knee']
    group_clean = group.copy()
    for part in parts_to_clean:
        if f'{part}_conf' in group_clean.columns:
            low_conf_mask = group_clean[f'{part}_conf'] < 0.5
            group_clean.loc[low_conf_mask, f'norm_{part}_x'] = np.nan
            group_clean.loc[low_conf_mask, f'norm_{part}_y'] = np.nan
    norm_cols = [col for col in group_clean.columns if col.startswith('norm_')]
    group_clean[norm_cols] = group_clean[norm_cols].interpolate(method='linear', limit_direction='both', limit=5)

    # 1. 지표 계산
    group_with_metrics = calculate_hundred_metrics(group_clean)

    # --- [2. Active 구간 선정 (심플 로직)] ---
    # 다리가 엉덩이보다 높게(각도 > 5도) 올라간 구간만 선택
    # (0도 대신 5도를 쓰는 이유: 바닥에 누워있을 때의 미세한 떨림/노이즈 제외)
    active_frames = group_with_metrics[group_with_metrics['Leg_Height_Angle'] > 5].copy()

    if active_frames.empty: return None

    # 연속성 체크: 가장 긴 구간 선택 (잠깐 든 것 제외)
    active_frames['frame_diff'] = active_frames['Frame'].diff()
    active_frames['block_id'] = (active_frames['frame_diff'] > 5).cumsum()
    longest_block_id = active_frames.groupby('block_id').size().idxmax()
    final_active_frames = active_frames[active_frames['block_id'] == longest_block_id]

    if len(final_active_frames) < 30: return None # 1초 미만 무시

    # --- [3. 통계 산출 (최빈값 포함)] ---

    # (1) Dominant Angle (가장 오래 유지한 각도)
    dominant_leg_angle = get_dominant_angle(final_active_frames['Leg_Height_Angle'])
    dominant_core_angle = get_dominant_angle(final_active_frames['Core_Angle'])

    # (2) 유지율 (Consistency Score)
    leg_error_margin = 5.0
    frames_in_zone = final_active_frames[
        (final_active_frames['Leg_Height_Angle'] >= dominant_leg_angle - leg_error_margin) &
        (final_active_frames['Leg_Height_Angle'] <= dominant_leg_angle + leg_error_margin)
    ]
    consistency_score = (len(frames_in_zone) / len(final_active_frames)) * 100

    # (3) 기존 지표
    stability_score = final_active_frames['Leg_Height_Angle'].std()

    if len(final_active_frames['Frame'].unique()) > 1:
        slope, _ = np.polyfit(final_active_frames['Frame'], final_active_frames['Leg_Height_Angle'], 1)
    else:
        slope = 0

    start_frame = final_active_frames['Frame'].min()
    end_frame = final_active_frames['Frame'].max()

    return pd.Series({
        'Dominant_Leg_Angle': dominant_leg_angle,   # [핵심] 사용자가 의도한 각도 (최빈값)
        'Dominant_Core_Angle': dominant_core_angle,
        'Consistency_Score': consistency_score,     # 유지율
        'Leg_Stability_StdDev': stability_score,
        'Leg_Fatigue_Slope': slope,
        'Active_Frame_Count': len(final_active_frames),
        'Active_Start_Frame': start_frame,
        'Active_End_Frame': end_frame
    })

# --- 3. 메인 실행부 ---

drive_root = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/dataset/Hundred'
norm_folder_path = os.path.join(drive_root,'keypoint_Hundred',  'norm_x')
all_files = glob.glob(os.path.join(norm_folder_path, "norm_*.csv"))

output_path = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/analysis'

if not all_files:
    print(f"❌ '{norm_folder_path}' 경로에 'norm_*.csv' 파일이 없습니다.")
else:
    print(f"✅ 총 {len(all_files)}개의 파일을 분석합니다 (다리 각도 > 5도 기준).")

    results_list = []

    for i, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        if i % 10 == 0: print(f"  [{i+1}/{len(all_files)}] 처리 중...")

        try:
            clean_name = filename.replace('norm_', '').replace('.csv', '')
            parts = clean_name.split('_')

            place = parts[2]
            level = parts[5]
            person_id = parts[6]
            pose_name = parts[4]

            for part in parts:
                if part.startswith('actor'): person_id = part
                if part in ['고급', '중급', '초급']: level = part
                if part.lower() == 'hundred': pose_name = 'Hundred'

            if pose_name != 'Hundred': continue

            df_single_file = pd.read_csv(file_path)
            stability_summary = analyze_stability(df_single_file)

            if stability_summary is not None:
                stability_summary['Source_File'] = filename
                stability_summary['Person_ID'] = person_id
                stability_summary['Level'] = level
                stability_summary['Place'] = place
                results_list.append(stability_summary)

        except Exception as e:
            print(f"    ⚠️ {filename} 오류: {e}")

    if results_list:
        df_final_eda = pd.DataFrame(results_list)

        output_dir = os.path.join(output_path, "keypoint_Hundred_Analysis")
        save_path = os.path.join(output_dir, "EDA_Hundred_Angle_Mode_Report_1124.csv") # 파일명 변경

        os.makedirs(output_dir, exist_ok=True)
        df_final_eda.to_csv(save_path, index=False)

        print("-" * 50)
        print("✅ 분석 완료! (다리 각도 기반 최빈값 분석)")
        print(f"저장된 파일: {save_path}")
        print(df_final_eda.head())
    else:
        print("유효한 데이터가 없습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 총 117개의 파일을 분석합니다 (다리 각도 > 5도 기준).
  [1/117] 처리 중...
  [11/117] 처리 중...
  [21/117] 처리 중...
  [31/117] 처리 중...
  [41/117] 처리 중...
  [51/117] 처리 중...
  [61/117] 처리 중...
  [71/117] 처리 중...
  [81/117] 처리 중...
  [91/117] 처리 중...
  [101/117] 처리 중...
  [111/117] 처리 중...
--------------------------------------------------
✅ 분석 완료! (다리 각도 기반 최빈값 분석)
저장된 파일: /content/drive/MyDrive/shared_googledrive(Sessac Final Project)/analysis/keypoint_Hundred_Analysis/EDA_Hundred_Angle_Mode_Report_1124.csv
   Dominant_Leg_Angle  Dominant_Core_Angle  Consistency_Score  \
0                16.0                165.0          60.000000   
1                27.0                149.0          62.983425   
2                25.0                151.0          65.359477   
3                23.0                159.0          88.181818   
4                14.0                168.0          58.2

In [ ]:
import pandas as pd
import numpy as np

# 1. CSV 파일 불러오기 (파일명은 실제 파일로 변경하세요)
df = pd.read_csv('/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/analysis/keypoint_Hundred_Analysis/EDA_Hundred_Angle_Mode_Report_1124.csv')
df['Leg_Angle'] = df['Dominant_Leg_Angle']

print(f"--- [1] 전체 원본 데이터 ({len(df)}개) ---")
desc = df['Leg_Angle'].describe()
Q1 = desc['25%']
Q3 = desc['75%']
IQR = Q3 - Q1
Lower_Fence = Q1 - 1.5 * IQR
Upper_Fence = Q3 + 1.5 * IQR

print(f"Q1: {Q1:.2f} | Q3: {Q3:.2f}")
print(f"IQR 범위 (Boxplot): {Lower_Fence:.2f} ~ {Upper_Fence:.2f}")
print("-" * 50)


# ---------------------------------------------------------
# 방법 A: 도메인 지식 기반 제거 (물리적 필터링)
# 논리: "5도 미만은 바닥에 닿은 쉬는 시간이고, 80도 초과는 측정이 잘못된 것이다."
# ---------------------------------------------------------
filtered_df_A = df[(df['Leg_Angle'] >= 10) & (df['Leg_Angle'] <= 70)]

print(f"\n--- [2] 물리적 노이즈(10도 미만, 70도 초과) 제거 후 ({len(filtered_df_A)}개) ---")
desc_A = filtered_df_A['Leg_Angle'].describe()
Q1_A = desc_A['25%']
Q3_A = desc_A['75%']
IQR_A = Q3_A - Q1_A
Lower_A = Q1_A - 1.5 * IQR_A
Upper_A = Q3_A + 1.5 * IQR_A

print(f"재계산된 Q1: {Q1_A:.2f} | Q3: {Q3_A:.2f}")
print(f"재계산된 상/하한: {Lower_A:.2f} ~ {Upper_A:.2f}")
print("-" * 50)


# ---------------------------------------------------------
# 방법 B: 통계적 절사 (Trimming) - 상위/하위 10% 제거
# 논리: "양쪽 끝 10%의 극단적인 수행을 제외하고, '보편적인 수행'만 보겠다."
# ---------------------------------------------------------
q_low = df['Leg_Angle'].quantile(0.10) # 하위 10% 지점
q_high = df['Leg_Angle'].quantile(0.90) # 상위 10% 지점

filtered_df_B = df[(df['Leg_Angle'] >= q_low) & (df['Leg_Angle'] <= q_high)]

print(f"\n--- [3] 양쪽 끝 10% 절사(Trimming) 후 ({len(filtered_df_B)}개) ---")
desc_B = filtered_df_B['Leg_Angle'].describe()
Q1_B = desc_B['25%']
Q3_B = desc_B['75%']
IQR_B = Q3_B - Q1_B
Lower_B = Q1_B - 1.5 * IQR_B
Upper_B = Q3_B + 1.5 * IQR_B

print(f"절사 기준: {q_low:.2f}도 ~ {q_high:.2f}도 사이만 남김")
print(f"재계산된 Q1: {Q1_B:.2f} | Q3: {Q3_B:.2f}")
print(f"재계산된 상/하한: {Lower_B:.2f} ~ {Upper_B:.2f}")

--- [1] 전체 원본 데이터 (117개) ---
Q1: 18.00 | Q3: 53.00
IQR 범위 (Boxplot): -34.50 ~ 105.50
--------------------------------------------------

--- [2] 물리적 노이즈(10도 미만, 70도 초과) 제거 후 (110개) ---
재계산된 Q1: 18.25 | Q3: 53.00
재계산된 상/하한: -33.88 ~ 105.12
--------------------------------------------------

--- [3] 양쪽 끝 10% 절사(Trimming) 후 (99개) ---
절사 기준: 12.00도 ~ 58.00도 사이만 남김
재계산된 Q1: 18.00 | Q3: 51.00
재계산된 상/하한: -31.50 ~ 100.50


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Dominant_Leg_Angle    117 non-null    float64
 1   Dominant_Core_Angle   117 non-null    float64
 2   Consistency_Score     117 non-null    float64
 3   Leg_Stability_StdDev  117 non-null    float64
 4   Leg_Fatigue_Slope     117 non-null    float64
 5   Active_Frame_Count    117 non-null    float64
 6   Active_Start_Frame    117 non-null    float64
 7   Active_End_Frame      117 non-null    float64
 8   Source_File           117 non-null    object 
 9   Person_ID             117 non-null    object 
 10  Level                 117 non-null    object 
 11  Place                 117 non-null    object 
dtypes: float64(8), object(4)
memory usage: 11.1+ KB
